<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DATA-SET:-Chicago's-Food-Inspection" data-toc-modified-id="DATA-SET:-Chicago's-Food-Inspection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DATA SET: Chicago's Food Inspection</a></span></li><li><span><a href="#Data-Overview" data-toc-modified-id="Data-Overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Overview</a></span></li><li><span><a href="#Treatment-of-the-data" data-toc-modified-id="Treatment-of-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Treatment of the data</a></span><ul class="toc-item"><li><span><a href="#inspection_id" data-toc-modified-id="inspection_id-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>inspection_id</a></span></li><li><span><a href="#DBA-name" data-toc-modified-id="DBA-name-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>DBA name</a></span></li><li><span><a href="#AKA-Name" data-toc-modified-id="AKA-Name-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>AKA Name</a></span></li><li><span><a href="#License-Number" data-toc-modified-id="License-Number-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>License Number</a></span></li><li><span><a href="#Facility-Type" data-toc-modified-id="Facility-Type-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Facility Type</a></span></li><li><span><a href="#Risk" data-toc-modified-id="Risk-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Risk</a></span></li><li><span><a href="#Address" data-toc-modified-id="Address-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Address</a></span></li><li><span><a href="#City" data-toc-modified-id="City-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>City</a></span></li><li><span><a href="#State" data-toc-modified-id="State-3.9"><span class="toc-item-num">3.9&nbsp;&nbsp;</span>State</a></span></li><li><span><a href="#Location" data-toc-modified-id="Location-3.10"><span class="toc-item-num">3.10&nbsp;&nbsp;</span>Location</a></span></li><li><span><a href="#Inspection-type" data-toc-modified-id="Inspection-type-3.11"><span class="toc-item-num">3.11&nbsp;&nbsp;</span>Inspection type</a></span></li></ul></li><li><span><a href="#Data-Exploration" data-toc-modified-id="Data-Exploration-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Exploration</a></span><ul class="toc-item"><li><span><a href="#Missing-Data" data-toc-modified-id="Missing-Data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Missing Data</a></span></li><li><span><a href="#Inspections-by-type--of-business" data-toc-modified-id="Inspections-by-type--of-business-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Inspections by type  of business</a></span></li><li><span><a href="#Relationship-between-columns" data-toc-modified-id="Relationship-between-columns-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Relationship between columns</a></span></li><li><span><a href="#Where-are-the-inspections-located?" data-toc-modified-id="Where-are-the-inspections-located?-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Where are the inspections located?</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import pickle 
from matplotlib import pyplot as plt
import warnings
from pyjarowinkler import distance
import geopandas as gpd
import folium
import seaborn as sns
import pgeocode
from itertools import combinations
warnings.filterwarnings("ignore")

# DATA SET: Chicago's Food Inspection

Chicago's Food Inspection Dataset is derived from a larger initiative from the local authorities of Chicago to make Government's data publicly available to everyone. Other similar datasets are available at https://data.cityofchicago.org

This dataset, in particular, is generated from inspections of restaurants and other food establishments in Chicago from January 1, 2010, to the present. Inspections are performed by staff from the Chicago Department of Public Health’s Food Protection Program using a standardized procedure. The results of the inspection are inputted into a database, then reviewed and approved by a State of Illinois Licensed Environmental Health Practitioner (LEHP).

- __Data Owner__: Chicago Department of Public Health
- __Time Period__: 2010 - Present
- __Frequency__: This database was updated with information from new inspections each Friday.


# Data Overview

First of all, we are going to understand the dataset and its content. Initial analyis on the food-inspections dataset in Chicago follows below:

In [2]:
chicago_df = pd.read_csv('./data/food-inspections.csv', delimiter=',')
chicago_df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2320830,"THE HOXTON, CHICAGO","THE HOXTON, CHICAGO",2694640.0,Restaurant,Risk 2 (Medium),200 N GREEN ST,CHICAGO,IL,60607.0,...,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.885699,-87.648789,"{'latitude': '-87.64878908937915', 'longitude'...",NaN,NaN,NaN,NaN,NaN
1,2320831,OGDEN PLAZA INC.,OGDEN PLAZA INC.,2475982.0,Grocery Store,Risk 3 (Low),3459 W OGDEN AVE,CHICAGO,IL,60623.0,...,Out of Business,NaN,41.855266,-87.712402,"{'latitude': '-87.71240156240032', 'longitude'...",NaN,NaN,NaN,NaN,NaN
2,2320829,PLAZA FOOD AND LIQUOR,PLAZA FOOD AND LIQUOR,2689756.0,Grocery Store,Risk 3 (Low),3455-3459 S OGDEN AVE,CHICAGO,IL,60623.0,...,Not Ready,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2320813,PLAZA FOOD AND LIQUOR,PLAZA FOOD AND LIQUOR,2689757.0,Grocery Store,Risk 3 (Low),3455-3459 S OGDEN AVE,CHICAGO,IL,60623.0,...,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2320757,GADS HILL CENTER,GADS HILL CENTER,2698627.0,Daycare Above and Under 2 Years,Risk 1 (High),4255-4259 S ARCHER AVE,CHICAGO,IL,60632.0,...,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.816005,-87.700893,"{'latitude': '-87.70089338917239', 'longitude'...",NaN,NaN,NaN,NaN,NaN


In [3]:
chicago_df.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards'],
      dtype='object')

As we can see, this dataset has 22 different columns, of different type:
- **Integer columns**: `'Inspection ID'`, `'License #'`, `'Zip'`
- **Floating point columns**: `'Latitude'`, `'Longitude'`
- **Date columns**: `'Inspection Date'`
- **String Columns**: `'Results'`\*, `'DBA Name'`, `'Address'`, `'Risk'`\*, `'Facility Type'`\*, `'City'`\*, `'State'`\*, `'Location'`, `'AKA Name'`, `'Inspection Type'`\*, `'Violations'`^, where:
   - \*= Categorical
   - ^= Too many categories to be called categorical (142431)
- **Unknown data-type columns**: `'Wards'`, `'Community Areas'`, `'Zip Codes'`, `'Historical Wards 2003-2015'`, `'Census Tracts'`

The contents of the columns whose meaning is not immediate are explained below:

- __DBA__: ‘Doing business as.’ This is the legal name of the establishment.
- __AKA__: ‘Also known as.’ This is the name the public would know the establishment as.
- __License number__: This is a unique number assigned to the establishment for the purposes of licensing by the Department of Business Affairs and Consumer Protection.
- __Type of facility__: Each establishment is described by one of the following: bakery, banquet hall, candy store, caterer, coffee shop, day care center (for ages less than 2), day care center (for ages 2 – 6), day care center (combo, for ages less than 2 and 2 – 6 combined), gas station, Golden Diner, grocery store, hospital, long term care center(nursing home), liquor store, mobile food dispenser, restaurant, paleteria, school, shelter, tavern, social club, wholesaler, or Wrigley Field Rooftop.
- __Risk category of facility__: Each establishment is categorized as to its risk of adversely affecting the public’s health, with 1 being the highest and 3 the lowest. The frequency of inspection is tied to this risk, with risk 1 establishments inspected most frequently and risk 3 least frequently.
- __Street address, city, state and zip code of facility__: This data provides a full address for each business.
- __Inspection date__: This is the date the inspection occurred. A particular establishment is likely to have multiple inspections which are denoted by different inspection dates.
- __Inspection type__: An inspection can be one of the following types: 
 - *canvass*: the most common type of inspection performed at a frequency relative to the risk of the establishment.
 - *consultation*: when the inspection is done at the request of the owner prior to the opening of the establishment. 
 - *complaint*: when the inspection is done in response to a complaint against the establishment.
 - *license*: when the inspection is done as a requirement for the establishment to receive its license to operate.
 - *suspect food poisoning* when the inspection is done in response to one or more persons claiming to have gotten ill as a result of eating at the establishment (a specific type of complaintbased inspection).
 - *task-force inspection* when an inspection of a bar or tavern is done.
 
 Re-inspections can occur for most types of these inspections and are indicated as such.
- __Results__: An inspection can pass, pass with conditions or fail. Establishments receiving a ‘pass’ were found to have no critical or serious violations (violation number 1-14 and 15- 29, respectively). Establishments receiving a ‘pass with conditions’ were found to have critical or serious violations, but these were corrected during the inspection. Establishments receiving a ‘fail’ were found to have critical or serious violations that were not correctable during the inspection. An establishment receiving a ‘fail’ does not necessarily mean the establishment’s licensed is suspended. Establishments found to be out of business or not located are indicated as such.
- __Violations__: An establishment can receive one or more of 45 distinct violations (violation numbers 1-44 and 70). For each violation number listed for a given establishment, the requirement the establishment must meet in order for it to NOT receive a violation is noted, followed by a specific description of the findings that caused the violation to be issued.


In [4]:
# get basic min/max statistics to get a feeling of some column's values range
basic_stats = (pd.Series(chicago_df.min(), name='mins').to_frame()
               .join(pd.Series(chicago_df.max(), name='maxs')))
basic_stats.index.name = 'Column'
basic_stats

,mins,maxs
Column,,
Inspection ID,44247,2341211
DBA Name,"#1 CHINA EXPRESS, LTD.",vitino pizzeria
License #,0,1e+07
Address,,N2660 HAYTON RD
Zip,10014,60827
Inspection Date,2010-01-04T00:00:00.000,2019-10-31T00:00:00.000
Results,Business Not Located,Pass w/ Conditions
Latitude,41.6447,42.0211
Longitude,-87.9144,-87.5251


We observe there are 5 columns filled with NaN values: `Historical`, `Wards 2003-2015`, `Zip Codes`, `Community Areas`, `Census Tracts`, `Wards`. 

We are going to remove these columns from our data frame. From now on, we will only work with the dataframe where these colums have been removed (*'chicago_df_noNan'*).

In [5]:
chicago_df_noNan = chicago_df.drop(columns=["Historical Wards 2003-2015",
                                            "Zip Codes",
                                            "Community Areas",
                                            "Census Tracts",
                                            "Wards"])

In [6]:
chicago_df_noNan.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2320830,"THE HOXTON, CHICAGO","THE HOXTON, CHICAGO",2694640.0,Restaurant,Risk 2 (Medium),200 N GREEN ST,CHICAGO,IL,60607.0,2019-10-31T00:00:00.000,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.885699,-87.648789,"{'latitude': '-87.64878908937915', 'longitude'..."
1,2320831,OGDEN PLAZA INC.,OGDEN PLAZA INC.,2475982.0,Grocery Store,Risk 3 (Low),3459 W OGDEN AVE,CHICAGO,IL,60623.0,2019-10-31T00:00:00.000,Canvass,Out of Business,NaN,41.855266,-87.712402,"{'latitude': '-87.71240156240032', 'longitude'..."
2,2320829,PLAZA FOOD AND LIQUOR,PLAZA FOOD AND LIQUOR,2689756.0,Grocery Store,Risk 3 (Low),3455-3459 S OGDEN AVE,CHICAGO,IL,60623.0,2019-10-31T00:00:00.000,License,Not Ready,NaN,NaN,NaN,NaN
3,2320813,PLAZA FOOD AND LIQUOR,PLAZA FOOD AND LIQUOR,2689757.0,Grocery Store,Risk 3 (Low),3455-3459 S OGDEN AVE,CHICAGO,IL,60623.0,2019-10-31T00:00:00.000,License,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,NaN,NaN,NaN
4,2320757,GADS HILL CENTER,GADS HILL CENTER,2698627.0,Daycare Above and Under 2 Years,Risk 1 (High),4255-4259 S ARCHER AVE,CHICAGO,IL,60632.0,2019-10-30T00:00:00.000,License,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.816005,-87.700893,"{'latitude': '-87.70089338917239', 'longitude'..."


# Treatment of the data
In this part, the following columns of the data are examined and cleaned:
Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State'

## inspection_id
Some inspections have been inserted twice in the dataset and need to be deleted to avoid duplicates.

In [7]:
# Create more coding friendly coding names
og_columns = chicago_df_noNan.columns
columns = [
    'inspection_id', 'dba_name', 'aka_name', 'license', 
    'facility_type', 'risk', 'address', 'city', 'state', 
    'zip', 'inspection_date', 'inspection_type', 'results', 
    'violations', 'latitude', 'longitude', 'location'
]
chicago_df_noNan.columns = columns

In [8]:
inspection_id = chicago_df_noNan['inspection_id']

We first verify that no entry has a null id

In [9]:
sum(inspection_id.isnull())

0

Then, we check for duplicate entries

In [10]:
duplicated = chicago_df_noNan.duplicated(keep='first')

In [11]:
print(len(chicago_df_noNan))
print(np.sum(duplicated))
chicago_df_noNan = chicago_df_noNan.drop_duplicates()
print(len(chicago_df_noNan)) 

195116
129
194987


## DBA name

In [12]:
# Checking if there are any null entries
sum(chicago_df_noNan['dba_name'].isnull())

0

In [13]:
# Standardising the name of the chain by lowering the cases
print('Number of unique restaurants in the dataset when the names are case sensitive:', len(chicago_df_noNan['dba_name'].unique()))
chicago_df_noNan['dba_name'] = chicago_df_noNan['dba_name'].str.lower()
print('Number of unique restaurants when the names are not case sensitive:', len(chicago_df_noNan['dba_name'].unique()))

Number of unique restaurants in the dataset when the names are case sensitive: 27513
Number of unique restaurants when the names are not case sensitive: 27227


## AKA Name
In this part, we cast all aka names into lower case, and replace the null entries by their DBA names

In [14]:
chicago_df_noNan['aka_name'].unique()

array(['THE HOXTON, CHICAGO', 'OGDEN PLAZA INC.', 'PLAZA FOOD AND LIQUOR',
       ..., 'SAFAH FOOD & LIQUOR INC', 'RAINBOW GROCERY', 'MAKIA FOOD'],
      dtype=object)

In [15]:
print('Number of entries without an AKA name:', len(chicago_df_noNan[chicago_df_noNan['aka_name'].isnull()]))

Number of entries without an AKA name: 2449


In [16]:
# Replacing null entries with their dba name
temp0 = chicago_df_noNan[chicago_df_noNan['aka_name'].isnull()]
temp0['aka_name'] = temp0['dba_name']
temp1 = chicago_df_noNan[~chicago_df_noNan['aka_name'].isnull()]
chicago_df_noNan = temp1.append(temp0)

In [17]:
print('Number of unique names with case sensitivity:', len(chicago_df_noNan['aka_name'].unique()))
temp = chicago_df_noNan['aka_name'].str.lower()
print('Number of unique names without case sensitivity:', len(temp.unique()))
chicago_df_noNan['aka_name'] = chicago_df_noNan['aka_name'].str.lower()

Number of unique names with case sensitivity: 27181
Number of unique names without case sensitivity: 26705


## License Number
Some business don't have a license number. In this dataframe, we create a boolean column, indicating if the business has a license number or not

In [18]:
print('There are ', len(chicago_df_noNan[chicago_df_noNan['license'].isnull()]), 'entries without a license number')

There are  17 entries without a license number


In [19]:
chicago_df_noNan['has_license'] = ~chicago_df_noNan['license'].isnull()

## Facility Type
Some types of facilities only contain one establishment, either because it is a very niche category, or because it was entered in a way that the similar entries don't match because of spelling or specificity. In order to group those categories, we match the facility types grouping less than 40 establishments to the most similar category according to the Jaro Winkler distance. If none of the categories are a match (distance smaller than 0.70), we place them into misc (miscellaneous).

In [20]:
# Turn Nan values into a string to be able to operate on the column
# Standardising the facility types to lower case categories
chicago_df_noNan['facility_type']
len(chicago_df_noNan['facility_type'].unique())
chicago_df_noNan['facility_type'] = chicago_df_noNan['facility_type'].fillna('not available')
chicago_df_noNan['facility_type'] = chicago_df_noNan['facility_type'].str.lower()

In [21]:
len(chicago_df_noNan)

194987

In [22]:
print('Number of facility types before standardising:', len(chicago_df_noNan['facility_type'].unique()))

Number of facility types before standardising: 441


In [23]:
# get an idea of the facility types, before standardising
chicago_df_noNan['facility_type'].unique()[380:]

array(['bakery/ restaurant', 'mexican candy store', 'not for profit',
       'live butcher', 'meat market', 'drug treatment facility',
       'shared kitchen user (long trem)', 'restaurant/liquor', 'np-kiosk',
       'day care combo (1586)', 'liquor consumption on premises.',
       'mobil food 1315', 'watermelon house', 'assissted living',
       'nutrition store', 'daycare 1586', 'food pantry/church',
       'paleteria /icecream shop', 'bakery/restaurant',
       'unlicensed facility', '(convenience store)', 'peddler',
       'hooka bar', 'kids cafe',
       'prepackage meal distributor (1006 retail)',
       'gift/card shop with candy', '(gas station)',
       'coffee vending machine', 'drug store/w/ food', 'commiasary',
       'grocery/liquor', 'ice cream parlor', "farmer's market",
       'produce stand', 'produce vendor', 'pharmacy/grocery',
       'soup kitchen', 'hooka lounge', 'candy maker', 'tavern/liquor',
       'incubator', 'wholesale bakery', "kids cafe'",
       'exercis

In [24]:
facility_count = chicago_df_noNan['facility_type'].value_counts()

i = 0

facility_types = []
small_types = {}

# Retrieving the main categories, and identifying the smallest ones
for facility in facility_count.index:
    if facility_count[facility] > 40:
        facility_types.append(facility)
    elif facility_count[facility] <= 40:
        small_types[facility] = ''

# Matching the small categories to the principal ones
for small in small_types:
    distances = []
    for facility in facility_types:
        dist = distance.get_jaro_distance(small, facility)
        distances.append(dist)
    index = np.argmax(distances)
    if distances[index] > 0.70:
        small_types[small] = facility_types[index]
    else:
        small_types[small] = 'misc'


In [25]:
print('Minority categories and their match in the main category pool:')
print(small_types)

Minority categories and their match in the main category pool:
{'store': 'misc', 'church': 'misc', 'restaurant/bar': 'restaurant', 'rooftop': 'misc', 'ice cream shop': 'coffee shop', "1023 childern's services facility": "children's services facility", 'church kitchen': 'shared kitchen', 'commissary': 'misc', 'cooking school': 'charter school', 'bar': 'bakery', 'assisted living': 'misc', "1023-children's services facility": "children's services facility", 'pop-up establishment host-tier ii': 'misc', 'grocery & restaurant': 'grocery/restaurant', 'culinary school': 'charter school', 'restaurant/grocery store': 'restaurant', 'roof tops': 'misc', 'theater': 'shelter', 'mobile desserts vendor': 'mobile frozen desserts vendor', 'nursing home': 'misc', 'restaurant/gas station': 'restaurant', 'roof top': 'misc', 'grocery store/gas station': 'grocery store', 'gas station/mini mart': 'gas station', 'after school program': 'private school', 'wrigley roof top': 'misc', 'paleteria': 'cafeteria', 'gr

In [26]:
# Replacing the minority categories by majority ones in the dataframe
chicago_df_noNan['new_facility_type'] = chicago_df_noNan['facility_type']

def get_new_facility(x):
    if x not in small_types:
        return x
    else:
        return small_types[x]
    
chicago_df_noNan['new_facility_type'] = chicago_df_noNan['new_facility_type'].apply(get_new_facility)

In [27]:
print('Number of facilities in the dataset after matching minority types to majority types:', len(chicago_df_noNan['new_facility_type'].unique()))

Number of facilities in the dataset after matching minority types to majority types: 43


## Risk
Filled nan values by Risk -1 and change risk 'All' to risk 4 (All), so that we can extract the data numerically if we want later


In [28]:
chicago_df_noNan['risk'].unique()
chicago_df_noNan['risk'] = chicago_df_noNan['risk'].fillna('Risk -1 (None)')
chicago_df_noNan['risk'] = chicago_df_noNan['risk'].str.replace('All', 'Risk 4 (All)')

In [29]:
chicago_df_noNan['risk'].unique()

array(['Risk 2 (Medium)', 'Risk 3 (Low)', 'Risk 1 (High)',
       'Risk -1 (None)', 'Risk 4 (All)'], dtype=object)

## Address

In [30]:
chicago_df_noNan['city'].unique()

array(['CHICAGO', nan, 'chicago', 'Chicago', 'GRIFFITH', 'NEW YORK',
       'SCHAUMBURG', 'ELMHURST', 'NEW HOLSTEIN', 'ALGONQUIN', 'CCHICAGO',
       'NILES NILES', 'EVANSTON', 'CHICAGO.', 'CHESTNUT STREET',
       'LANSING', 'CHICAGOCHICAGO', 'WADSWORTH', 'WILMETTE', 'WHEATON',
       'CHICAGOHICAGO', 'ROSEMONT', 'CHicago', 'CALUMET CITY',
       'PLAINFIELD', 'HIGHLAND PARK', 'PALOS PARK', 'ELK GROVE VILLAGE',
       'BRIDGEVIEW', 'CICERO', 'OAK PARK', 'MAYWOOD', 'LAKE BLUFF',
       '312CHICAGO', 'SCHILLER PARK', 'SKOKIE', 'BEDFORD PARK',
       'BANNOCKBURNDEERFIELD', 'CHCICAGO', 'BLOOMINGDALE', 'Norridge',
       'CHARLES A HAYES', 'CHCHICAGO', 'CHICAGOI', 'SUMMIT',
       'OOLYMPIA FIELDS', 'WESTMONT', 'CHICAGO HEIGHTS', 'JUSTICE',
       'TINLEY PARK', 'LOMBARD', 'EAST HAZEL CREST', 'COUNTRY CLUB HILLS',
       'STREAMWOOD', 'BOLINGBROOK', 'INACTIVE', 'BERWYN', 'BURNHAM',
       'DES PLAINES', 'LAKE ZURICH', 'OLYMPIA FIELDS', 'OAK LAWN',
       'BLUE ISLAND', 'GLENCOE', 'FRANKFO

In [31]:
def split_address(address):
    liste = address.upper().split(' ')
    nr = liste[0]
    cardinal = liste[1]
    reste = ' '.join(liste[2:])
    liste = nr + ('*') + cardinal + '*' + reste
    return liste

In [32]:
temp = chicago_df_noNan['address'].apply(split_address)
temp = temp.str.split('*', expand=True)


## City

In [33]:
print(len(chicago_df_noNan['city'].unique()))
chicago_df_noNan['city'] = chicago_df_noNan['city'].str.lower()
print(len(chicago_df_noNan['city'].unique()))

72
67


In [34]:
chicago_df_noNan['city'].unique()
chicago_df_noNan['city'] = chicago_df_noNan['city'].replace('cchicago', 'chicago').replace('chicago.', 'chicago')
chicago_df_noNan['city'] = chicago_df_noNan['city'].replace('chicagochicago', 'chicago')
chicago_df_noNan['city'] = chicago_df_noNan['city'].replace('chicagohicago', 'chicago')
chicago_df_noNan['city'] = chicago_df_noNan['city'].replace('312chicago', 'chicago').replace('chicagoi', 'chicago')
chicago_df_noNan['city'] = chicago_df_noNan['city'].replace('chchicago', 'chicago')
chicago_df_noNan['city'] = chicago_df_noNan['city'].replace('chcicago', 'chicago')
chicago_df_noNan['city'] = chicago_df_noNan['city'].fillna('chicago')

In [35]:
np.sort(chicago_df_noNan['city'].unique())

array(['algonquin', 'alsip', 'bannockburndeerfield', 'bedford park',
       'berwyn', 'bloomingdale', 'blue island', 'bolingbrook',
       'bridgeview', 'broadview', 'burnham', 'calumet city',
       'charles a hayes', 'chestnut street', 'chicago', 'chicago heights',
       'cicero', 'country club hills', 'des plaines', 'east hazel crest',
       'elk grove village', 'elmhurst', 'evanston', 'evergreen park',
       'frankfort', 'glencoe', 'griffith', 'highland park', 'inactive',
       'justice', 'lake bluff', 'lake zurich', 'lansing', 'lombard',
       'maywood', 'naperville', 'new holstein', 'new york', 'niles niles',
       'norridge', 'oak lawn', 'oak park', 'olympia fields',
       'oolympia fields', 'palos park', 'plainfield', 'rosemont',
       'schaumburg', 'schiller park', 'skokie', 'streamwood', 'summit',
       'tinley park', 'wadsworth', 'westmont', 'wheaton', 'wilmette',
       'worth'], dtype=object)

In [36]:
cities = pd.read_csv('./data/listsChicago.csv', sep=';', header=None)
cities[0] = cities[0].str.lower()
cities = cities[0].values

In [37]:
outs = []
ins = []
for city in chicago_df_noNan['city'].unique():
    if city not in cities:
        outs.append(city)
        chicago_df_noNan = chicago_df_noNan[chicago_df_noNan['city'] != city]
    else:
        ins.append(city)

In [38]:
print('Here are the real cities from Chicago!')
print(ins)

Here are the real cities from Chicago!
['chicago', 'bridgeview', 'cicero', 'oak park', 'maywood', 'bedford park', 'berwyn', 'oak lawn', 'broadview', 'evergreen park']


## State
 We make sure that all our remaining entries are from Chicago in Illinois! 

In [39]:
print(chicago_df_noNan['state'].unique())

['IL' nan]


In [40]:
chicago_df_noNan['state'] = np.where(chicago_df_noNan['state'].isnull(), 'IL', chicago_df_noNan['state'])
print(chicago_df_noNan['state'].unique())

['IL']


## Location

In [41]:
missing_longitude_indices = chicago_df_noNan['longitude'].isna()

In [42]:
np.sum(missing_longitude_indices)

556

In [43]:
chicago_df_noNan[missing_longitude_indices][['latitude', 'location']].isna().sum()

latitude    556
location    556
dtype: int64

We can conclude that all location attributes are missing together

In [44]:
# Find how many of the rows with missing location attributes have missing zip code also
np.sum(chicago_df_noNan['longitude'].isna() & chicago_df_noNan['zip'].isna())

3

In [45]:
# Define methods that performs geocoding using the zipcode

def geocode_zip_code(zip_code):
    nomi = pgeocode.Nominatim('us')
    nomi_query = nomi.query_postal_code(int(zip_code))
    return pd.Series([nomi_query.latitude, nomi_query.longitude])

In [46]:
# Perform geocoding using the zipcode
missing_longitude_indices_with_zip = chicago_df_noNan['longitude'].isna() & chicago_df_noNan['zip'].notna()
missing_locations = chicago_df_noNan.loc[missing_longitude_indices_with_zip, 'zip'].apply(geocode_zip_code)

In [47]:
chicago_df_noNan.loc[missing_longitude_indices_with_zip, 'latitude'] = missing_locations[0]
chicago_df_noNan.loc[missing_longitude_indices_with_zip, 'longitude'] = missing_locations[1]

In [48]:
missing_longitude_indices = chicago_df_noNan['latitude'].isna()

In [49]:
np.sum(missing_longitude_indices)

3

We have filled most of the missing locations by using geocoding using the zipcode which would help us define the areas, other geocoders using the address leading to more accurate locations exist but they either require keys such as google maps or require better address format such as open street maps which does not work for of all our examples

In [50]:
# Create a groupedby data frame on the name and aggeragate to find the count of missing longitude and total counts for each group
exp_df = chicago_df_noNan[['longitude', 'dba_name']].groupby('dba_name').agg({"longitude": [lambda x: x.isnull().sum()], 'dba_name':'count'})

In [51]:
# rename column for more intutive view
exp_df.columns.set_levels(['nulls_count', 'count'],level=1,inplace=True)

In [52]:
# Identify how many entries with missing locations have other entries for the same place with the location not missing

bool_1 = (exp_df['longitude']['nulls_count'] != 0)
bool_2 = (exp_df['dba_name']['count'] != exp_df['longitude']['nulls_count'])

np.sum(exp_df[ bool_1.values & bool_2.values]['longitude']['nulls_count'])

2.0

In [53]:
# use restaurants with multiple entries where some have their location fields to fill the same restaurant entries with missing location

missing_location_names = chicago_df_noNan.loc[missing_longitude_indices]['dba_name']

for index, name in missing_location_names.items():
    similar_entries = chicago_df_noNan[(chicago_df_noNan['dba_name'] == name) & (chicago_df_noNan['dba_name'].notna())] 
    chicago_df_noNan.at[index, 'latitude'] = similar_entries.iloc[0]['latitude']
    chicago_df_noNan.at[index, 'longitude'] = similar_entries.iloc[0]['longitude']
    chicago_df_noNan.at[index, 'zip'] = similar_entries.iloc[0]['zip']

In [54]:
missing_longitude_indices = chicago_df_noNan['latitude'].isna()

In [55]:
np.sum(missing_longitude_indices)

1

In [56]:
chicago_df_noNan.columns

Index(['inspection_id', 'dba_name', 'aka_name', 'license', 'facility_type',
       'risk', 'address', 'city', 'state', 'zip', 'inspection_date',
       'inspection_type', 'results', 'violations', 'latitude', 'longitude',
       'location', 'has_license', 'new_facility_type'],
      dtype='object')

## Inspection type

Here we clean the inspection type data by:
1. Casefolding
1. Removing digits because the examples were for different task forces which are very similar
1. Removing stopwords that made sense in our dataset
1. Correcting spelling
1. Lemmatization of words

In [57]:
from nltk.corpus import wordnet as wn
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [58]:
# lowercase everything
chicago_df_noNan['inspection_type'] = chicago_df_noNan['inspection_type'].str.lower()

In [59]:
# Check similar entries
check_series = chicago_df_noNan['inspection_type']
iteratable = combinations(check_series.unique(), r=2)
inconsistent_pairs = [pair for pair in iteratable if distance.get_jaro_distance(str(pair[0]), str(pair[1])) > 0.7]
inconsistent_pairs

[('license', 'license re-inspection'),
 ('license', 'license-task force'),
 ('license', 'license task 1474'),
 ('license', 'license renewal inspection for daycare'),
 ('license', 'license renewal for daycare'),
 ('license', 'license request'),
 ('license', 'license task force / not -for-profit clu'),
 ('license', 'license task force / not -for-profit club'),
 ('license', 'license/not ready'),
 ('license', 'license daycare 1586'),
 ('license', 'license canceled by owner'),
 ('license', 'license consultation'),
 ('license', 'license wrong address'),
 ('canvass', 'canvass re-inspection'),
 ('canvass', 'canvass special events'),
 ('canvass', 'canvass school/special event'),
 ('canvass', 'canvas'),
 ('canvass', 'canvass re inspection of close up'),
 ('canvass', 'canvass/special event'),
 ('canvass', 'canvass for rib fest'),
 ('license re-inspection', 'canvass re-inspection'),
 ('license re-inspection', 'complaint re-inspection'),
 ('license re-inspection', 'recent inspection'),
 ('license r

In [60]:
# Identify methods for cleaning the text

# Remove digit sequences
def remove_digits(list_):
    list_ = [i for i in list_ if not i.isdigit()] 
    return list_

# Remove stopwords
def remove_stopwords(list_, verbrose=False):
    # Set of stop words that made sense with our data the predefined removed words we may need
    stop_words = ['by', 'for', 'and', 'from', 'of']
    filtered_sentence = [w for w in list_ if not w in stop_words]
    if verbrose and len(list_) > len(filtered_sentence):
        print("Removed Stopwords")
        print(list_)
        print(filtered_sentence)
    return filtered_sentence

# Identify sentence with misspelling
def is_misspelled(list_):
    stop_words = set(stopwords.words('english'))
    for word in list_:
        if len(wn.synsets(word)) == 0 and word not in stop_words:
            return True
    return False

# Correct sentences with misspellings
def correct_misspellings(list_):
    # Correction dictionary that made sense for our data because the dataset is using abbreviations we do not want to remove
    correct_dict = {'reinspection': ['re', 'inspection'], 'clu': ['club'], 'questionaire': ['questionaire'], 'taskforce': ['task', 'force']}
    misspelled = list(set(list_).intersection(correct_dict.keys()))
    for word in misspelled:
        index = list_.index(word)
        list_[index:index+1] = correct_dict[word]
        
    return list_    

# Lemmatizing words in sentences
def lemmatization(list_):
    lemmatizer = WordNetLemmatizer() 
    return [lemmatizer.lemmatize(word) for word in list_]

# Define jaccard similarity to compare cleaned strings
def jaccard_similarity(string1, string2):
    list1 = string1.split(' ')
    list2 = string2.split(' ')
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [61]:
# Inspect problems on the unique items first to identify misspellings
tokenizer = RegexpTokenizer(r'\w+')
unique_inspection_type = pd.DataFrame(chicago_df_noNan['inspection_type'].unique(), columns=['inspection_type'])
unique_inspection_type['inspection_type'] = unique_inspection_type['inspection_type'].map(str)
unique_inspection_type['inspection_type'] = unique_inspection_type['inspection_type'].map(tokenizer.tokenize)
unique_inspection_type['inspection_type'] = unique_inspection_type['inspection_type'].map(remove_digits)
unique_inspection_type['inspection_type'] = unique_inspection_type['inspection_type'].map(remove_stopwords)
unique_inspection_type['mispelled'] = unique_inspection_type['inspection_type'].apply(is_misspelled)

In [62]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(unique_inspection_type[unique_inspection_type['mispelled']])

                             inspection_type  mispelled
28              [pre, license, consultation]       True
39                   [license, reinspection]       True
42                                     [sfp]       True
44                          [sfp, complaint]       True
48                [sfp, recently, inspected]       True
54                               [taskforce]       True
55  [license, task, force, not, profit, clu]       True
59                                    [citf]       True
70                     [haccp, questionaire]       True
71                         [out, ofbusiness]       True
72      [close, up, complaint, reinspection]       True
77              [reinspection, hour, notice]       True
78                            [reinspection]       True
80                      [canvass, rib, fest]       True
84         [liqour, task, force, not, ready]       True


In [ ]:
# Clean full dataset
chicago_df_noNan['inspection_type'] = chicago_df_noNan['inspection_type'].map(str)
chicago_df_noNan['inspection_type'] = chicago_df_noNan['inspection_type'].map(tokenizer.tokenize)
chicago_df_noNan['inspection_type'] = chicago_df_noNan['inspection_type'].map(remove_digits)
chicago_df_noNan['inspection_type'] = chicago_df_noNan['inspection_type'].map(remove_stopwords)
chicago_df_noNan['mispelled'] = chicago_df_noNan['inspection_type'].apply(is_misspelled)
chicago_df_noNan[chicago_df_noNan['mispelled']]['inspection_type'] = chicago_df_noNan[chicago_df_noNan['mispelled']]['inspection_type'].map(correct_misspellings)
chicago_df_noNan['mispelled'] = chicago_df_noNan['inspection_type'].apply(is_misspelled)
chicago_df_noNan['inspection_type'] = chicago_df_noNan['inspection_type'].map(lemmatization)
chicago_df_noNan['inspection_type'] = chicago_df_noNan['inspection_type'].map(' '.join)

In [ ]:
# Look for similar inspection_types
iteratable = combinations(chicago_df_noNan['inspection_type'].unique(), r=2)
inconsistent_pairs = [pair for pair in iteratable if jaccard_similarity(str(pair[0]), str(pair[1])) > 0.7]
inconsistent_pairs

In [ ]:
# Drop flag column after cleaning
chicago_df_noNan = chicago_df_noNan.drop(columns=['mispelled'], axis=1)

Now after cleaning there still seem to be different entries that look very similar, However, since we do not know if they will help or not, we will them for now.

# Data Exploration

## Missing Data

Let us first evaluate how our dataset looks like, after the cleaning.

In [ ]:
# get missing data percentages
total = chicago_df_noNan.isnull().sum().sort_values(ascending=False)
percentage = (100*(chicago_df_noNan.isnull().sum()/chicago_df_noNan.isnull().count() 
                  ).sort_values(ascending=False))
missing_data = pd.concat([total, percentage], axis=1, keys=['Total', 'Percentage'])
missing_data.index.name = 'Column'
print("Number of total entries in dataframe: {}".format(chicago_df_noNan.shape[0]))
missing_data

From this table, we can obtain some interesting data. 

For instance, from a **total number of 195212** reports of inspections submitted, 143452 *(= 195212-51760)* violations were found out. This corresponds to **a rate of 73.48% violations found** in the inspections made.

## Inspections by type  of business

First, we'll want to see how inspections are distributed by type of business:

In [ ]:
# Inspections by type of restaurant
nInspectionsByType = chicago_df_noNan["new_facility_type"].value_counts()
nInspectionsByType.head()

In [ ]:
nInspectionsByType.plot.bar()
plt.xticks(fontsize=7)
plt.title("Inspections by (treated) type of facility")
plt.ylabel("(Logarithmic) number of inspections")
plt.yscale('log')
plt.show()

The plot above presents the number of inspections by type of business - one pertinent observation would be that most of the inspections correspond to restaurants.

## Relationship between columns

Let us see how the columns of the dataset are related. 

To start, we will plot the correlations between every pair of columns in order to see how they can be connected to each other.

In [ ]:
sns.set()
sns.pairplot(chicago_df_noNan.dropna())
plt.show()

From the previous plot, we can extract some interesting results:
- The plot (`Latitude`, `Longitude`) gives us a sense of the city's shape, as it becomes a plot of the data points in space (for comparison, an map of the can also be found further bellow). Furthermore, both `Longitude` and `Latitude` are approximately naturally distributed in their value range. 
- The `Zip` address is concentrated at 60000, which is not surprising as Chicago's zip codes are concentrated at around 60600. The histogram plot easily evidencies the presence of at least two outliers, due to it's unexpected x-axis value range - these may be restaurants in the outskirts of the city.
- We would expect a uniform distribution for the `Inspection ID`, but the plotted histogram may reflect a national-scale license attribution system.
- Finally, inspections seem to be pretty uniformly dispersed throughout the different `License #` values, whose histogram also evidencies one outlier.

After looking at this plot, we may ask which of these variable are linearly correlated. And this is the next question we are going to answer from our dataset.

## Where are the inspections located?

Previously, we saw the plot of `('Latitude', 'Longitude')` already creates a basic map with the location of the businesses within the city.

Let's formalize this map and understand better where the businesses subject to inspections are located

In [ ]:
# New DataFrame with the zip codes and #inspections per zip code
businessesByZip = chicago_df_noNan.groupby('zip')[['inspection_id']].count()
businessesByZip = businessesByZip.rename(columns={'inspection_id':'nInspections'})
businessesByZip.reset_index(inplace=True)
businessesByZip['zip'] = businessesByZip['zip'].astype(int).astype('str')
businessesByZip.head(3)

In [ ]:
m = folium.Map(location=[41.8781, -87.6298])
zip_geo = f'./data/zipCodesBoundaries.geojson'

folium.Choropleth(
    geo_data=zip_geo,
    name='choropleth',
    data=businessesByZip,
    columns=['zip','nInspections'],
    key_on='feature.properties.zip',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Businesses per zip'
).add_to(m)

folium.LayerControl().add_to(m)

m

This map contains the number of inspections made by zip code area, and reflects how the majority of inspections are located in downtown Chicago